Also from Po, a Star Wars blaster battle puzzle:

In Star Wars battles, sometimes a severely outnumbered force emerges victorious through superior skill. You panic when you see a group of nine stormtroopers coming at you from very far away in the distance. Fortunately, they are notoriously inaccurate with their blaster fire, with only a 0.1 percent chance of hitting you with each of their shots. You and each stormtrooper fire blasters at the same rate, but you are $K$ times as likely to hit your target with each shot. Furthermore, the stormtroopers walk in a tight formation, and can therefore create a larger target area. Specifically, if there are N stormtroopers left, your chance of hitting one of them is $(K⋅\sqrt{N})/1000$. Each shot has an independent probability of hitting and immediately taking out its target. For approximately what value of $K$ is this a fair match between you and the stormtroopers (where you have 50 percent chance of blasting all of them)?

# [RobShearer solution](https://gist.github.com/rvcx/808d1b669eaa326d9fa2b7cebcf8c359)

In [14]:
from math import sqrt
from random import random

def simulate(n, k, base_rate):
    while n:
        # each storm trooper shoots
        for i in range(n):
            if random() < base_rate:
                return False
        # you shoot
        if random() < k * base_rate * sqrt(n):
            n -= 1
    return True

def sample(size, func):
     return sum(1.0 for x in range(size) if func()) / size

def target(tval, min, max, steps, func):
    for i in range(steps):
        probe = (min + max) / 2.0
        val = func(probe)
        print(probe, val, tval)
        if val < tval:
            min = probe
        else:
            max = probe
    return probe

print(target(0.5, 1, 1000, 10,lambda k: sample(1000,lambda: simulate(9, k, 0.001))))

500.5 0.946 0.5
250.75 0.924 0.5
125.875 0.873 0.5
63.4375 0.755 0.5
32.21875 0.542 0.5
16.609375 0.347 0.5
24.4140625 0.48 0.5
28.31640625 0.543 0.5
26.365234375 0.484 0.5
27.3408203125 0.504 0.5
27.3408203125


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
Ks = [1,2,6,10,20,40,80,100]
N = np.arange(1,10)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])